## Deliverable 2. Create a Customer Travel Destinations Map.

In [48]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [49]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,55.90,90,10,9.01,clear sky
1,1,Necochea,AR,-38.5473,-58.7368,51.04,95,1,8.14,clear sky
2,2,Tuktoyaktuk,CA,69.4541,-133.0374,34.72,76,100,8.46,overcast clouds
3,3,Khasan,RU,42.4308,130.6434,63.30,64,25,5.37,scattered clouds
4,4,San Quintin,MX,30.4833,-115.9500,72.30,71,5,10.83,clear sky


In [50]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [51]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,8,Hilo,US,19.7297,-155.0900,84.27,65,1,10.36,clear sky
9,9,Bundaberg,AU,-24.8500,152.3500,77.29,46,40,10.00,scattered clouds
19,19,Vanimo,PG,-2.6741,141.3028,78.66,86,100,4.90,overcast clouds
20,20,Puerto Carreno,CO,6.1890,-67.4859,76.24,91,100,5.21,overcast clouds
25,25,Butaritari,KI,3.0707,172.7902,82.63,76,6,15.55,light rain
...,...,...,...,...,...,...,...,...,...,...
678,678,Papara,BR,-3.9839,-38.7273,79.20,93,43,4.50,scattered clouds
680,680,Bandhi,PK,26.5861,68.3006,82.27,51,55,7.74,broken clouds
683,683,Madang,PG,-5.2246,145.7966,82.31,72,97,4.18,overcast clouds
684,684,Cozumel,MX,20.5083,-86.9458,82.99,84,75,3.00,broken clouds


In [52]:
# 4a. Determine if there are any empty rows.
if preferred_cities_df.isnull().values.any() == True:
    print("Null values present in the Preferred cities list")

In [53]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
if preferred_cities_df.isnull().values.any() == True:
    preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_clean_df = preferred_cities_df

In [54]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = None
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Hilo,US,84.27,clear sky,19.7297,-155.0900,None
9,Bundaberg,AU,77.29,scattered clouds,-24.8500,152.3500,None
19,Vanimo,PG,78.66,overcast clouds,-2.6741,141.3028,None
20,Puerto Carreno,CO,76.24,overcast clouds,6.1890,-67.4859,None
25,Butaritari,KI,82.63,light rain,3.0707,172.7902,None
...,...,...,...,...,...,...,...
678,Papara,BR,79.20,scattered clouds,-3.9839,-38.7273,None
680,Bandhi,PK,82.27,broken clouds,26.5861,68.3006,None
683,Madang,PG,82.31,overcast clouds,-5.2246,145.7966,None
684,Cozumel,MX,82.99,broken clouds,20.5083,-86.9458,None


In [55]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found... skipping.{index}")

Hotel not found... skipping.163
Hotel not found... skipping.171
Hotel not found... skipping.179
Hotel not found... skipping.233
Hotel not found... skipping.260
Hotel not found... skipping.281
Hotel not found... skipping.302
Hotel not found... skipping.566
Hotel not found... skipping.584
Hotel not found... skipping.659


In [56]:
# 7. Drop the rows where there is no Hotel Name.
hotel_clean_df = hotel_df.dropna()
hotel_clean_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Hilo,US,84.27,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
9,Bundaberg,AU,77.29,scattered clouds,-24.8500,152.3500,Takalvan Motel
19,Vanimo,PG,78.66,overcast clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
20,Puerto Carreno,CO,76.24,overcast clouds,6.1890,-67.4859,Casa Hotel Victoria
25,Butaritari,KI,82.63,light rain,3.0707,172.7902,Isles Sunset Lodge
...,...,...,...,...,...,...,...
678,Papara,BR,79.20,scattered clouds,-3.9839,-38.7273,Romin
680,Bandhi,PK,82.27,broken clouds,26.5861,68.3006,"A,M Malik sweet š Bakers"
683,Madang,PG,82.31,overcast clouds,-5.2246,145.7966,Comfort Inn Hotel
684,Cozumel,MX,82.99,broken clouds,20.5083,-86.9458,Cozumel Palace® All Inclusive Resort


In [58]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_clean_df.to_csv(output_data_file, index_label="City_ID")

In [61]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_clean_df[["Lat", "Lng"]]

In [62]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))